In [2]:
import numpy as np
import copy
import time
from vm_generator import *

In [3]:
overload_data, unoverload_data = InstanceGenerator(5,100)

#objective: select minimum vm mem lists

capacity = 100 #physical node CPU threshold
weight = [i[0] for i in overload_data[:]] #vm cpu usage -> weight
profit = [i[1] for i in overload_data[:]]  #vm mem usage -> value

unoverloaded_bin = [i[0] for i in unoverload_data[:]]


In [4]:
#greedy to select vm list

def greedy_select_vm(profit, weight, capacity):
    selected_vm_cpu = []
    for p, w in zip(profit, weight):
        mem_sorted, cSortByMem = (list(i) for i in zip(*sorted(zip(p,w)))) #sorted mem size and weight sorted by mem size
        
        current_weight = sum(cSortByMem)
        for w in cSortByMem:
            if current_weight <= capacity:
                break

            selected_vm_cpu.append(w)
            current_weight -= w

    print(f"Selected VM numbers: {len(selected_vm_cpu)}")
    print(f"Selected VM CPU: {selected_vm_cpu}")
    return selected_vm_cpu


In [5]:
#knapsack to select reserved vm
def knapSack(W: int, wt, val, n):
    reserved = []
    m = [[0 for x in range(W + 1)] for x in range(n + 1)]
    
    for i in range(n + 1):
        for w in range (W + 1):
            if i == 0 or w == 0:
                m[i][w] = 0
            elif wt[i-1] > w:
                m[i][w] = m[i-1][w]
            else:
                m[i][w] = max(m[i-1][w],m[i-1][w-wt[i-1]]+val[i-1])
            
    result = m[n][W]
    # print(result)
    
    
    w = W
    for i in range(n,0,-1):
        if result <= 0:
            break
        if result == m[i-1][w]:
            continue
        else:
            # print(i-1,wt[i-1])
            reserved.append(wt[i-1])
            result = result - val[i-1]
            w = w-wt[i-1]
    return reserved

In [6]:
def pack(values, maxValue):
    migrateCounter = newMachineCounter = 0

    values = sorted(values, reverse=True)
    bins = copy.deepcopy(unoverloaded_bin)

    for item in values:
        # Try to fit item into a bin
        min_space = float('inf')
        min_index = -1
        for index, bin in enumerate(bins):
            #find minimum space and enough capacity to accomodate
            if sum(bin) + item <= maxValue and (maxValue-sum(bin)) < min_space:
                min_space = maxValue-sum(bin)
                min_index = index

        if min_index == -1:
            # item didn't fit into any bin, start a new bin
            bin = []
            bin.append(item)
            migrateCounter += 1
            bins.append(bin)
            newMachineCounter += 1
        else:
            bins[min_index].append(item)
            migrateCounter += 1

    return bins, migrateCounter, newMachineCounter


def packAndShow(aList, maxValue):

        bins, migrateCounter, newMachineCounter = pack(aList, maxValue)
        
        print(f"Migration Times: {migrateCounter}")
        print(f"New Machine Times: {newMachineCounter}")

        print ('Solution using', len(bins), 'bins:')
        for bin in bins:
            print (bin)

In [7]:

if __name__ == '__main__':
    
    start = time.time()
    selected_vm_cpu = greedy_select_vm(profit, weight, capacity)
    packAndShow(selected_vm_cpu, 100)
    end = time.time()

    print(f"time: {end-start}")


Selected VM numbers: 40
Selected VM CPU: [24, 17, 30, 21, 12, 27, 22, 18, 24, 22, 15, 28, 27, 29, 17, 22, 29, 25, 25, 29, 9, 17, 20, 25, 20, 29, 29, 16, 17, 19, 26, 16, 24, 22, 30, 9, 28, 19, 30, 14]
Migration Times: 40
New Machine Times: 0
Solution using 60 bins:
[23, 10, 12, 13, 14, 28]
[10, 25, 14, 11, 15, 25]
[10, 16, 12, 14, 19, 29]
[19, 13, 12, 17, 12, 27]
[16, 14, 10, 15, 17, 28]
[10, 22, 10, 21, 11, 26]
[12, 17, 14, 18, 11, 14]
[17, 14, 10, 12, 20, 27]
[16, 12, 19, 11, 16, 22]
[16, 11, 13, 12, 13]
[14, 16, 10, 19, 13]
[13, 16, 10, 18, 15]
[14, 14, 10, 12, 12]
[17, 14, 13, 17, 13, 21]
[14, 15, 16, 10, 11]
[20, 15, 10, 12, 10]
[11, 10, 13, 10, 13]
[12, 20, 20, 11, 11, 20]
[15, 11, 12, 23, 12, 17, 9]
[25, 10, 10, 14, 13]
[26, 10, 10, 15, 12, 17]
[10, 23, 18, 10, 13, 20]
[30, 10, 10, 10, 15, 25]
[10, 11, 21, 11, 18, 29]
[12, 12, 19, 15, 12, 30]
[13, 24, 12, 12, 12, 17]
[22, 10, 17, 12, 12, 16]
[10, 12, 11, 18, 19, 30]
[17, 12, 15, 11, 11]
[15, 10, 28, 11, 10, 19]
[13, 11, 11, 18, 1

In [8]:

if __name__ == '__main__':

    start = time.time()
    selected_vm_cpu = []
    for w, p in zip(weight, profit):
        #all - reserved = need to be migrated vms
        reserved = knapSack(capacity, w, p, len(p))
        migrate = list(w)
        for vm in reserved:
            migrate.remove(vm)
        selected_vm_cpu += migrate
        
    print(f"Selected VM numbers: {len(selected_vm_cpu)}")
    print(f"Selected VM CPU: {selected_vm_cpu}")
    packAndShow(selected_vm_cpu, 100)

    end = time.time()

    print(f"time: {end-start}")



Selected VM numbers: 36
Selected VM CPU: [24, 30, 21, 12, 27, 22, 18, 24, 22, 15, 28, 29, 17, 22, 29, 25, 25, 29, 9, 17, 20, 25, 20, 29, 29, 16, 17, 19, 26, 24, 30, 9, 28, 19, 30, 14]
Migration Times: 36
New Machine Times: 0
Solution using 60 bins:
[23, 10, 12, 13, 14, 28]
[10, 25, 14, 11, 15, 25]
[10, 16, 12, 14, 19, 29]
[19, 13, 12, 17, 12, 27]
[16, 14, 10, 15, 17, 28]
[10, 22, 10, 21, 11, 26]
[12, 17, 14, 18, 11]
[17, 14, 10, 12, 20, 17]
[16, 12, 19, 11, 16, 21]
[16, 11, 13, 12, 13]
[14, 16, 10, 19, 13]
[13, 16, 10, 18, 15]
[14, 14, 10, 12, 12]
[17, 14, 13, 17, 13, 20]
[14, 15, 16, 10, 11]
[20, 15, 10, 12, 10]
[11, 10, 13, 10, 13]
[12, 20, 20, 11, 11, 20]
[15, 11, 12, 23, 12, 16]
[25, 10, 10, 14, 13]
[26, 10, 10, 15, 12, 15, 12]
[10, 23, 18, 10, 13, 19]
[30, 10, 10, 10, 15, 25]
[10, 11, 21, 11, 18, 29]
[12, 12, 19, 15, 12, 30]
[13, 24, 12, 12, 12, 14]
[22, 10, 17, 12, 12]
[10, 12, 11, 18, 19, 30]
[17, 12, 15, 11, 11]
[15, 10, 28, 11, 10, 19]
[13, 11, 11, 18, 15]
[11, 10, 14, 20, 15,